# 0. Install Dependencies

In [ ]:
!pip install tensorflow==2.5.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install numpy

# 1. Test Random Environment with OpenAI Gym

In [37]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [56]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            # reward = -1
            reward = -1 * (abs(self.state - 38) / 100) 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [57]:
env = ShowerEnv()

In [61]:
env.observation_space.sample()

array([48.66014], dtype=float32)

In [62]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:4.940000000000002
Episode:2 Score:10.970000000000006
Episode:3 Score:-3.9399999999999986
Episode:4 Score:17.700000000000006
Episode:5 Score:28.150000000000006
Episode:6 Score:2.45
Episode:7 Score:11.280000000000017
Episode:8 Score:23.890000000000004
Episode:9 Score:31.220000000000002
Episode:10 Score:3.0600000000000076


# 2. Create a Deep Learning Model with Keras

In [64]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [63]:
states = env.observation_space.shape
actions = env.action_space.n

In [78]:
actions

3

In [79]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [80]:
del model 

NameError: name 'model' is not defined

In [81]:
model = build_model(states, actions)

In [82]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [83]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [84]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [85]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 0.2037
166 episodes - episode_reward: 12.289 [-15.690, 43.630] - loss: 0.399 - mae: 5.286 - mean_q: 5.779

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 0.4785
167 episodes - episode_reward: 28.560 [-4.640, 52.850] - loss: 2.352 - mae: 7.692 - mean_q: 16.314

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 72s 7ms/step - reward: 0.6237
167 episodes - episode_reward: 37.423 [7.350, 54.900] - loss: 6.696 - mae: 12.409 - mean_q: 28.639

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 73s 7ms/step - reward: 0.6604
166 episodes - episode_reward: 39.591 [19.820, 53.870] - loss: 9.563 - mae: 14.986 - mean_q: 34.469

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: 0.6669
done, took 359

In [88]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 58.980, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 58.980, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 58.980, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 58.980, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st

In [90]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 58.980, steps: 60
Episode 3: reward: 58.980, steps: 60
Episode 4: reward: 58.980, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 58.980, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 58.980, steps: 60
Episode 15: reward: 58.980, steps: 60


# 4. Reloading Agent from Memory

In [34]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

NameError: name 'dqn' is not defined

In [31]:
del model
del dqn
del env

In [9]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [10]:
dqn.load_weights('dqn_weights.h5f')

In [11]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
